<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [2]:
#!pip install psycopg2
#!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org --trusted-host pypi.python.org psycopg2
import pandas as pd
import psycopg2

In [33]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [34]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [5]:
#Print 10 lines of the df

n=10
query = f'''
SELECT *    
FROM sql.pokemon
LIMIT {n}
'''

In [7]:
# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
df = pd.read_sql_query(query, connection)
df

# не забываем закрыть соединение после окончания работы
connection.close()

,id,name,type1,type2,hp,attack,defense,speed
0,1,Bulbasaur,Grass,Poison,45,49,49,45
1,2,Ivysaur,Grass,Poison,60,62,63,60
2,3,Venusaur,Grass,Poison,80,82,83,80
3,4,VenusaurMega Venusaur,Grass,Poison,80,100,123,80
4,7,Charizard,Fire,Flying,78,84,78,100
5,8,CharizardMega Charizard X,Fire,Dragon,78,130,111,100
6,9,CharizardMega Charizard Y,Fire,Flying,78,104,78,100
7,16,Butterfree,Bug,Flying,60,45,50,70
8,17,Weedle,Bug,Poison,40,35,30,50
9,18,Kakuna,Bug,Poison,45,25,50,35


In [41]:
query_3_1 = f'''
SELECT *    
  FROM vacancies
'''

In [55]:
vacancy_df = pd.read_sql_query(query_3_1, connection)
print('Количество вакансий -', vacancy_df['id'].nunique())

Количество вакансий - 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [47]:
query_3_2 = f'''
SELECT *    
  FROM employers
'''

In [48]:
employers_df = pd.read_sql_query(query_3_2, connection)
print('Количество работодателей -', employers_df['id'].nunique())

Количество работодателей - 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [49]:
query_3_3 = f'''
SELECT *    
  FROM areas
'''

In [50]:
areas_df = pd.read_sql_query(query_3_3, connection)
print('Количество регионов -', areas_df['id'].nunique())

Количество регионов - 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [53]:
query_3_4 = f'''
SELECT *    
  FROM industries
'''

In [54]:
industry_df = pd.read_sql_query(query_3_4, connection)
print('Количество сфер деятельности -', industry_df['id'].nunique())

Количество сфер деятельности - 294


***

Из представленной выше статистики можно сделать следующие выводы:
* База данных содержит обширный набор данных, который включает в себя:
    * 49197 уникальных вакансий от 23501 уникальных работодателей.
    * Вышеперечисленные вакансии представленны в 294-х различных сферах деятельности и расположены в 1362-х регионах.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [64]:
query_3_5 = f'''
SELECT 
COUNT(v.id) cnt,
a.name  area   
  FROM vacancies v
  JOIN areas a ON v.area_id = a.id
  GROUP BY a.id 
ORDER BY  cnt DESC
'''

In [69]:
query_3_5_df = pd.read_sql_query(query_3_5, connection)
print('Пятёрку лидеров по количеству вакансий:\n',  query_3_5_df.head(5))

Пятёрку лидеров по количеству вакансий:
     cnt             area
0  5333           Москва
1  2851  Санкт-Петербург
2  2112            Минск
3  2006      Новосибирск
4  1892           Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [80]:
query_3_6 = f'''
SELECT 
id,
salary_from,
salary_to  
  FROM vacancies v
   WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
'''

In [81]:
query_3_6_df = pd.read_sql_query(query_3_6, connection)
print('Количество вакансий у которых заполнено хотя бы одно из двух полей с зарплатой -',  query_3_6_df.shape[0])

Количество вакансий у которых заполнено хотя бы одно из двух полей с зарплатой - 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [84]:
query_3_7 = f'''
SELECT 
AVG(salary_from) salary_bottom,
AVG(salary_to) salary_top
  FROM vacancies v
'''

In [91]:
query_3_7_df = pd.read_sql_query(query_3_7, connection)
print('Среднее значение для нижней границы зарплаты -',  round(query_3_7_df['salary_bottom'][0], 0))
print('Среднее значение для верхней границы зарплаты -',  round(query_3_7_df['salary_top'][0], 0))

Среднее значение для нижней границы зарплаты - 71065.0
Среднее значение для верхней границы зарплаты - 110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [92]:
query_3_8 = f'''
SELECT 
COUNT(*) num_vacancies,
employment, 
schedule
  FROM vacancies v
  GROUP BY employment, schedule
  ORDER BY num_vacancies DESC
'''

In [115]:
query_3_8_df = pd.read_sql_query(query_3_8, connection)
query_3_8_df[query_3_8_df['num_vacancies']==query_3_8_df['num_vacancies'].nlargest(2)[1]][['employment','schedule']]

,employment,schedule
1,Полная занятость,Удаленная работа


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [117]:
query_3_9 = f'''
SELECT 
COUNT(*) num_vacancies,
experience
  FROM vacancies v
  GROUP BY experience
  ORDER BY num_vacancies ASC
'''

In [118]:
query_3_9_df = pd.read_sql_query(query_3_9, connection)
print('Требуемый опыт работы в порядке возрастания: \n',  query_3_9_df['experience'].iloc[:4])


Требуемый опыт работы в порядке возрастания: 
 0           Более 6 лет
1             Нет опыта
2         От 3 до 6 лет
3    От 1 года до 3 лет
Name: experience, dtype: object


***

In [ ]:
# выводы по детальному анализу вакансий

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [ ]:
# текст запроса

In [ ]:
# результат запроса

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [ ]:
# текст запроса

In [ ]:
# результат запроса

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [ ]:
# текст запроса

In [ ]:
# результат запроса

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [ ]:
# текст запроса

In [ ]:
# результат запроса

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [ ]:
# текст запроса

In [ ]:
# результат запроса

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [ ]:
# текст запроса

In [ ]:
# результат запроса

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [ ]:
# код для получения списка городов-милионников

In [ ]:
# текст запроса

In [ ]:
# результат запроса

***

In [ ]:
# выводы по анализу работодателей

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [ ]:
# текст запроса

In [ ]:
# результат запроса

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [ ]:
# текст запроса

In [ ]:
# результат запроса

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
# текст запроса

In [ ]:
# результат запроса

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
# текст запроса

In [ ]:
# результат запроса

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
# текст запроса

In [ ]:
# результат запроса

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [ ]:
# текст запроса

In [ ]:
# результат запроса

***

In [ ]:
# выводы по предметному анализу

# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования